In [1]:
import os
import numpy as np
import shutil
from shutil import copyfile
import random
import math
import zipfile
import matplotlib.pyplot as plt
np.warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from tensorflow.keras.models import load_model
from Models import VGG, Resnet

Using TensorFlow backend.


In [2]:
# if not os.path.exists('./ICIAR2018_BACH_Challenge.zip'):
#   !wget https://bibview.ir/Data/ICIAR2018_BACH_Challenge.zip
# else:
#   print("File ICIAR2018_BACH_Challenge already exists.")
# if not os.path.exists('./ICIAR2018_BACH_Challenge_TestDataset.zip'):
#   !wget https://bibview.ir/Data/ICIAR2018_BACH_Challenge_TestDataset.zip
# else:
#   print("File ICIAR2018_BACH_Challenge_TestDataset already exists.")


# if not os.path.exists('./ICIAR2018_BACH_Challenge'):
#   with zipfile.ZipFile('ICIAR2018_BACH_Challenge.zip') as file:
#     file.extractall()
# else:
#   print("File ICIAR2018_BACH_Challenge already extracted.")

# if not os.path.exists('./ICIAR2018_BACH_Challenge_TestDataset'):
#   with zipfile.ZipFile('ICIAR2018_BACH_Challenge_TestDataset.zip') as file:
#     file.extractall()
# else:
#   print("File ICIAR2018_BACH_Challenge_TestDataset already extracted.")

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
IMG_WIDTH, IMG_HEIGHT = 299, 299

POOLING_KERNEL_SIZE = 3
NUM_CLASSES = 4
num_epochs = 500

TRAIN_TO_TEST_RATIO = 0.9
DATA_DIR = './dataset/ICIAR2018_BACH_Challenge/Photos/'
CLASS_NAMES = ['Invasive', 'InSitu', 'Normal', 'Benign']
Train_dir = './dataset/ICIAR2018_BACH_Challenge/Photos/generator/Train/'
Test_dir = './dataset/ICIAR2018_BACH_Challenge/Photos/generator/Test/'

In [5]:
# for directory_name in os.listdir(DATA_DIR):
#     if directory_name in CLASS_NAMES:
#         class_path = os.path.join(DATA_DIR, directory_name)
#         img_file_names = []
#         for img_name in os.listdir(class_path):
#             if img_name.split(".")[1] == 'tif':
#                 img_path = os.path.join(class_path, img_name)
#                 img_file_names.append(img_name)
#         random.shuffle(img_file_names)
#         train_file_names = img_file_names[:int(math.floor(TRAIN_TO_TEST_RATIO * len(img_file_names)))]
#         test_file_names = img_file_names[int(math.floor(TRAIN_TO_TEST_RATIO * len(img_file_names))):]
        
#         class_train_path = os.path.join(Train_dir, directory_name)
#         class_test_path = os.path.join(Test_dir, directory_name)
        
#         if os.path.exists(class_train_path):
#             shutil.rmtree(class_train_path)
#         if os.path.exists(class_test_path):
#             shutil.rmtree(class_test_path)
        
#         os.mkdir(class_train_path)
#         os.mkdir(class_test_path)
        
#         for train_img_name in train_file_names:
#             train_img_path = os.path.join(class_train_path, train_img_name)
#             copyfile(os.path.join(class_path, train_img_name), train_img_path)
#             print("Copied train file %s" % train_img_name)
#         for test_img_name in test_file_names:
#             test_img_path = os.path.join(class_test_path, test_img_name)
#             copyfile(os.path.join(class_path, test_img_name), test_img_path)
#             print("Copied test file %s" % test_img_name)

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    tf.keras.layers.MaxPooling2D(POOLING_KERNEL_SIZE, POOLING_KERNEL_SIZE),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(POOLING_KERNEL_SIZE, POOLING_KERNEL_SIZE),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(POOLING_KERNEL_SIZE, POOLING_KERNEL_SIZE),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# vgg_model = VGG(model_name='my_model',
#             input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
#             input_layer_name='input_layer',
#             num_classes=NUM_CLASSES,
#            )

# model = vgg_model.construct_net()

model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 297, 297, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(Train_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT))

Found 360 images belonging to 4 classes.


In [8]:
validation_datagen = ImageDataGenerator(rescale=1./255,
                                        rotation_range=40,
                                        width_shift_range=0.3, 
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_generator = validation_datagen.flow_from_directory(Test_dir,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT))

Found 40 images belonging to 4 classes.


In [9]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model/', 
    verbose=1,
    save_weights_only=False,
    period=20,
)

cp_best_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model/best/', 
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
)

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=num_epochs,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks=[cp_best_callback, cp_callback]
                             )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 12 steps, validate for 2 steps
Epoch 1/500
11/12 [==========================>...] - ETA: 0s - loss: 1.3940 - acc: 0.2500
Epoch 00001: val_loss improved from inf to 1.38253, saving model to ./model/best/
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./model/best/assets
12/12 [==============================] - 12s 988ms/step - loss: 1.3919 - acc: 0.2556 - val_loss: 1.3825 - val_acc: 0.2500
Epoch 2/500
11/12 [==========================>...] - ETA: 0s - loss: 1.3756 - acc: 0.2530
Epoch 00002: val_loss improved from 1.38253 to 1.36902, saving model to ./model/best/
INFO:tensorflow:Assets written to: ./model/best/assets
12/12 [==============================] - 11s 934ms/step - loss: 1.3768 - acc: 0.2500 - val_loss: 1.3690 - val_acc: 0.2500
Epoch 3/500
11/12 [==========================>...]

Epoch 24/500
11/12 [==========================>...] - ETA: 0s - loss: 1.1615 - acc: 0.5000
Epoch 00024: val_loss did not improve from 1.17181
12/12 [==============================] - 11s 879ms/step - loss: 1.1761 - acc: 0.5028 - val_loss: 1.3029 - val_acc: 0.4500
Epoch 25/500
11/12 [==========================>...] - ETA: 0s - loss: 1.1586 - acc: 0.5000
Epoch 00025: val_loss did not improve from 1.17181
12/12 [==============================] - 11s 876ms/step - loss: 1.1461 - acc: 0.5083 - val_loss: 1.2249 - val_acc: 0.4000
Epoch 26/500
11/12 [==========================>...] - ETA: 0s - loss: 1.1263 - acc: 0.5213
Epoch 00026: val_loss did not improve from 1.17181
12/12 [==============================] - 11s 883ms/step - loss: 1.1248 - acc: 0.5139 - val_loss: 1.2123 - val_acc: 0.4000
Epoch 27/500
11/12 [==========================>...] - ETA: 0s - loss: 1.1405 - acc: 0.5183
Epoch 00027: val_loss did not improve from 1.17181
12/12 [==============================] - 11s 877ms/step - loss: 1.

11/12 [==========================>...] - ETA: 0s - loss: 1.0430 - acc: 0.5244
Epoch 00054: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 876ms/step - loss: 1.0471 - acc: 0.5194 - val_loss: 1.2761 - val_acc: 0.5750
Epoch 55/500
11/12 [==========================>...] - ETA: 0s - loss: 1.0234 - acc: 0.5540
Epoch 00055: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 879ms/step - loss: 1.0272 - acc: 0.5556 - val_loss: 1.3920 - val_acc: 0.5000
Epoch 56/500
11/12 [==========================>...] - ETA: 0s - loss: 1.0384 - acc: 0.5396
Epoch 00056: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 879ms/step - loss: 1.0301 - acc: 0.5472 - val_loss: 1.2235 - val_acc: 0.5250
Epoch 57/500
11/12 [==========================>...] - ETA: 0s - loss: 1.0138 - acc: 0.5671
Epoch 00057: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 882ms/step - loss: 1.0211 - acc: 0

Epoch 85/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9917 - acc: 0.5335
Epoch 00085: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 879ms/step - loss: 0.9921 - acc: 0.5361 - val_loss: 1.2364 - val_acc: 0.4750
Epoch 86/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9945 - acc: 0.5762
Epoch 00086: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 878ms/step - loss: 0.9935 - acc: 0.5778 - val_loss: 1.3998 - val_acc: 0.5500
Epoch 87/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9985 - acc: 0.5823
Epoch 00087: val_loss did not improve from 1.15661
12/12 [==============================] - 10s 875ms/step - loss: 0.9952 - acc: 0.5778 - val_loss: 1.3497 - val_acc: 0.5750
Epoch 88/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9738 - acc: 0.5854
Epoch 00088: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 881ms/step - loss: 0.

Epoch 116/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9355 - acc: 0.6037
Epoch 00116: val_loss did not improve from 1.15661
12/12 [==============================] - 11s 877ms/step - loss: 0.9339 - acc: 0.6056 - val_loss: 1.2089 - val_acc: 0.5000
Epoch 117/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9054 - acc: 0.6037
Epoch 00117: val_loss did not improve from 1.15661
12/12 [==============================] - 10s 874ms/step - loss: 0.9154 - acc: 0.5917 - val_loss: 1.2351 - val_acc: 0.5250
Epoch 118/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9582 - acc: 0.5762
Epoch 00118: val_loss did not improve from 1.15661
12/12 [==============================] - 10s 874ms/step - loss: 0.9631 - acc: 0.5722 - val_loss: 1.3421 - val_acc: 0.4750
Epoch 119/500
11/12 [==========================>...] - ETA: 0s - loss: 0.9634 - acc: 0.5579
Epoch 00119: val_loss did not improve from 1.15661
12/12 [==============================] - 10s 872ms/step - loss